In [1]:
! apt-get install default-jre
! java-version
! pip install h2o

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core ca-certificates-java dbus
  default-jre-headless fontconfig fonts-dejavu-extra gtk-update-icon-cache
  hicolor-icon-theme humanity-icon-theme java-common libapparmor1 libasound2
  libasound2-data libasyncns0 libatk-bridge2.0-0 libatk-wrapper-java
  libatk-wrapper-java-jni libatk1.0-0 libatk1.0-data libatspi2.0-0
  libavahi-client3 libavahi-common-data libavahi-common3 libcairo2 libcroco3
  libcups2 libdatrie1 libdbus-1-3 libdrm-amdgpu1 libdrm-common libdrm-intel1
  libdrm-nouveau2 libdrm-radeon1 libdrm2 libelf1 libflac8 libfontenc1
  libgail-common libgail18 libgdk-pixbuf2.0-0 libgdk-pixbuf2.0-bin
  libgdk-pixbuf2.0-common libgif7 libgl1-mesa-dri libgl1-mesa-glx
  libglapi-mesa libgraphite2-3 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libharfbuzz0b libjbig0 liblcms2-2 libllvm5.0 libnspr4 libnss3 libogg0


In [2]:
from google.colab import files
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_171"; OpenJDK Runtime Environment (build 1.8.0_171-8u171-b11-0ubuntu0.17.10.1-b11); OpenJDK 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpcvavfdpx
  JVM stdout: /tmp/tmpcvavfdpx/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpcvavfdpx/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.8
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_unknownUser_xgs549
H2O cluster total nodes:,1
H2O cluster free memory:,2.827 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


# Football scores:

Data set complexo!! Uma série temporal formada por três principais tipos de dados:

* Placar dos jogos e estatíticas;
* apostas antes do jogo;
* "força" do time antes de cada jogo

Será apresentado um problema de regressão logística (binomial classification): Estimar se o time da casa ganhará ou não.

Temos duas opções: usar os dados com informação das apostas ou não.
Vamos usar a métrica AUC por ser um problema de classificação binomial.


## Data set final:

#### HS1:
The shots by the current home side, in their previous match (whether they were
the home or away side in that previous match).
#### AS1
The shots by the current away side, in their previous match.
#### HST1/AST1
Shots on target in the previous match.
#### HF1/AF1
Fouls in the previous match.
#### HC1/AC1
Corners in the previous match.
#### HY1/AY1
Yellow cards in the previous match.
#### HR1/AR1
Red cards in the previous match.
#### res1H/res1A
The last result for the home and away sides: –1, 0 or +1.
#### res5H/res5A
Average of the last 5 results: –1.0 to +1.0.
#### res20H/res20A
Average of the last 20 results: –1.0 to +1.0.

### Missing Data (Again)
Decision tree machine-learning algorithms cope fairly well with missing data, but
deep learning will struggle a bit

I will train two models, the first with all fields ( x ), the second excluding the odds data
( xNoOdds )

In [0]:
path_train = "https://raw.githubusercontent.com/DarrenCook/h2o/bk/datasets/football.train.csv"
path_valid = "https://raw.githubusercontent.com/DarrenCook/h2o/bk/datasets/football.valid.csv"
path_test = "https://raw.githubusercontent.com/DarrenCook/h2o/bk/datasets/football.test.csv"

football_train = h2o.import_file(path_train)
football_valid = h2o.import_file(path_valid)
football_test = h2o.import_file(path_test)

football_train["HomeWin"] = (football_train["FTR"] == "H").asfactor()
football_valid["HomeWin"] = (football_valid["FTR"] == "H").asfactor()
football_test["HomeWin"] = (football_test["FTR"] == "H").asfactor()

football_train["ScoreDraw"] = ((football_train["FTHG"] > 0) & (football_train["FTHG"] == football_train["FTAG"])).asfactor()

football_valid["ScoreDraw"] = ((football_valid["FTHG"] > 0) & (football_valid["FTHG"] == football_valid["FTAG"])).asfactor()

football_test["ScoreDraw"] = ((football_test["FTHG"] > 0) & (football_test["FTHG"] == football_test["FTAG"])).asfactor()

statFields = [
  "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR",
  "HS", "AS", "HST", "AST", "HF", "AF",
  "HC", "AC", "HY", "AY", "HR", "AR",
  "HomeWin", "ScoreDraw"
  ]
ignoreFields = ["Date", "HomeTeam", "AwayTeam"] + statFields

x = [i for i in football_train.names if i not in ignoreFields]

xNoOdds = [
  "Div", "HS1", "AS1", "HST1", "AST1",
  "HF1", "AF1", "HC1", "AC1", "HY1", "AY1", "HR1", "AR1",
  "res1H", "res1A", "res5H", "res5A", "res20H", "res20A"
  ]

#y = "FTR"  #3-value multinomial
#y = "ScoreDraw"  #Unbalanced binomial
y = "HomeWin" #Balanced binomial